In [3]:
import pandas as pd
import numpy as np
import requests
import re
import urllib.parse
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from pathlib import Path
import json

In [11]:
#Mcdonalds Big Mac Index
Mcdf = pd.read_json("big-mac-index-by-state-2024.json")  # Read JSON file directly
print(Mcdf.head())

        state  BigMacIndexByState2023
0     Alabama                    3.99
1      Alaska                    4.87
2     Arizona                    4.43
3    Arkansas                    3.95
4  California                    5.11


In [13]:
#Minimum Wage data
#excel to dataframe
minwagedf = pd.read_excel('statistic_id238997_us-minimum-wage-2024-by-state.xlsx')